<a href="https://colab.research.google.com/github/subha9542/Datamining/blob/master/Data_mining_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations 
eps = np.finfo(float).eps
from numpy import log2 as log
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [5]:
!pwd
df = pd.read_csv("HW5.csv")
df.head

/content


<bound method NDFrame.head of    department  status     age    salary  count
0       sales  senior  31..35  46k..50k     30
1       sales  junior  26..30  26k..30k     40
2       sales  junior  31..35  31k..35k     40
3     systems  junior  21..25  46k..50k     20
4     systems  senior  31..35  66k..70k      5
5     systems  junior  26..30  46k..50k      3
6     systems  senior  41..45  66k..70k      3
7   marketing  senior  36..40  46k..50k     10
8   marketing  junior  31..35  41k..45k      4
9   secretary  senior  46..50  36k..40k      4
10  secretary  junior  26..30  26k..30k      6>

In [12]:
#https://medium.com/@rakendd/decision-tree-from-scratch-9e23bcfb4928
def entropy(df,target_attribute):
    entropy_node = 0  #Initialize Entropy
    values = df[target_attribute].unique() 
    for value in values:
        fraction = df.groupby(['status'])['count'].agg('sum')[value]/sum(df['count'])
        entropy_node += -fraction*np.log2(fraction)
    print("entropy: ",round(entropy_node,4))
    return round(entropy_node,4)

def info_gain(df,attribute,target_attribute):
    target_variables = df[target_attribute].unique() 
    variables = df[attribute].unique()    
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            attribute_num = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
            num = attribute_num['count'][attribute_num[attribute]==variable][attribute_num[target_attribute]==target_variable].sum()
            attribute_den = df.groupby([attribute],as_index=False)['count'].agg('sum')
            den = attribute_den['count'][attribute_den[attribute]==variable].sum()
            fraction = num/(den+eps)  
            entropy_each_feature += fraction*log(fraction+eps) 
        fraction2 = den/df['count'].sum()
        entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste
    print("entropy_",attribute,":",round(entropy_attribute,4))
    return round(entropy(df,'status') - entropy_attribute,4)

print("Info_gain = ","using department :",info_gain(df,'department','status'))
print("Info_gain = ","using age :",info_gain(df,'age','status'))
print("Info_gain = ","using salary :",info_gain(df,'salary','status'))

entropy_ department : 0.8504
entropy:  0.899
Info_gain =  using department : 0.0486
entropy_ age : 0.4743
entropy:  0.899
Info_gain =  using age : 0.4247
entropy_ salary : 0.3615
entropy:  0.899
Info_gain =  using salary : 0.5375


In [14]:
def split_info(df,attribute):
    values = df[attribute].unique() 
    split_info = 0
    for value in values:
        fraction = df.groupby([attribute])['count'].agg('sum')[value]/sum(df['count'])
        split_info += -fraction*np.log2(fraction)
    print("split_info_",attribute, split_info)
    return round(split_info,4)
print("Gain_ratio_","department",round(info_gain(df,'department','status')/split_info(df,'department'),4))
print("Gain_ratio_","age",round(info_gain(df,'age','status')/split_info(df,'age'),4))
print("Gain_ratio_","salary",round(info_gain(df,'salary','status')/split_info(df,'salary'),4))

entropy_ department : 0.8504
entropy:  0.899
split_info_ department 1.3902500689889323
Gain_ratio_ department 0.035
entropy_ age : 0.4743
entropy:  0.899
split_info_ age 1.878248324449327
Gain_ratio_ age 0.2261
entropy_ salary : 0.3615
entropy:  0.899
split_info_ salary 2.0115973114905925
Gain_ratio_ salary 0.2672


In [8]:
def get_list(df,attribute):
  items = df[attribute].unique() 
  #print(items)
  from itertools import combinations
  tmp = []
  for j in range(1,len(items)):
    for i in combinations(items, j):
      tmp.append(i)
  return list(set(tmp))
get_list(df,'department')


[('marketing',),
 ('systems',),
 ('systems', 'marketing'),
 ('sales', 'secretary'),
 ('sales', 'marketing'),
 ('sales', 'systems', 'secretary'),
 ('systems', 'secretary'),
 ('sales', 'systems'),
 ('systems', 'marketing', 'secretary'),
 ('sales', 'marketing', 'secretary'),
 ('secretary',),
 ('sales', 'systems', 'marketing'),
 ('marketing', 'secretary'),
 ('sales',)]

In [ ]:
def gini(df,attribute,target_attribute):
    target_variables = df[target_attribute].unique() 
    all_variables = df[attribute].unique()  
    entropy_attribute = 0
    variables = get_list(df,attribute)
    min = sys.maxsize
    split = ''
    for variable in variables:
        variable = list(variable)
        entropy_each_feature = 0
        attribute_num = df.groupby([attribute],as_index=False)['count'].agg('sum')
        num1 = 0
        for i in variable:
            num1 += attribute_num['count'][attribute_num[attribute] == i].sum()
        #print(num1)
        #for target_variable in target_variables:
        attribute_num2 = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
        num2 = 0
        for i in variable:
          num2 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='junior'].sum()
        num3 = 0
        for i in variable:
          num3 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='senior'].sum()
        #print(num1,num2,num3)
        #print(variable)
        remaining_variables = li_dif = [i for i in all_variables if i not in variable] 
        num4 = 0
        for i in remaining_variables:
            num4 += attribute_num['count'][attribute_num[attribute] == i].sum()
        #print(num1)
        #for target_variable in target_variables:
        attribute_num2 = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
        num5 = 0
        for i in remaining_variables:
          num5 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='junior'].sum()
        num6 = 0
        for i in remaining_variables:
          num6 += attribute_num2['count'][attribute_num2[attribute]==i][attribute_num2[target_attribute]=='senior'].sum()
        #print(num4,num5,num6)
        #print(remaining_variables)
        gini_index = (num1*(1 - (num2/num1)**2 - (num3/num1)**2) +  num4*(1 - (num5/num4)**2 - (num6/num4)**2))/sum(df['count'])
        if gini_index < min:
            min = gini_index
            split = variable
        print("gini split of",variable,round(gini_index,4))
    #print(min,split)
    #return round(entropy(df,'status') - entropy_attribute,4)
gini(df,'department','status')
gini(df,'age','status')
gini(df,'salary','status')  

In [23]:
def probability(df,attribute,target_attribute):
    variables = df[attribute].unique() 
    target_variables = df[target_attribute].unique() 
    for i in target_variables:
      for j in variables:
        num = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
        den = df.groupby(['status'],as_index=False)['count'].agg('sum')
        p_a_t = num['count'][num[attribute]==j][num[target_attribute]==i].sum()/den['count'][den['status'] == i].sum()
        print("Probability of",attribute,j,"given",i,round(p_a_t,4))

probability(df,'department','status')
probability(df,'age','status')
probability(df,'salary','status')

Probability of department sales given senior 0.5769
Probability of department systems given senior 0.1538
Probability of department marketing given senior 0.1923
Probability of department secretary given senior 0.0769
Probability of department sales given junior 0.708
Probability of department systems given junior 0.2035
Probability of department marketing given junior 0.0354
Probability of department secretary given junior 0.0531
Probability of age 31..35 given senior 0.6731
Probability of age 26..30 given senior 0.0
Probability of age 21..25 given senior 0.0
Probability of age 41..45 given senior 0.0577
Probability of age 36..40 given senior 0.1923
Probability of age 46..50 given senior 0.0769
Probability of age 31..35 given junior 0.3894
Probability of age 26..30 given junior 0.4336
Probability of age 21..25 given junior 0.177
Probability of age 41..45 given junior 0.0
Probability of age 36..40 given junior 0.0
Probability of age 46..50 given junior 0.0
Probability of salary 46k..50

In [48]:
def get_split_value(df,attribute,value):
    variables = df[attribute].unique() 
    for i in variables:
        j,k = i.split('..')
        if value >= int(j.rstrip('k')) and value <= int(k.strip('k')):
            return i
print(get_split_value(df,'salary',35))

31k..35k


In [54]:
def single_probability(df,attribute,value,target):
    num = df.groupby(['status',attribute],as_index=False)['count'].agg('sum')
    den = df.groupby(['status'],as_index=False)['count'].agg('sum')
    p_a_t = num['count'][num[attribute]==value][num['status']==target].sum()/den['count'][den['status'] == target].sum()
    return p_a_t

def def_status(df,department_attribute,age_attribute,salary_attribute):
      age = get_split_value(df,'age',age_attribute)
      salary = get_split_value(df,'salary',salary_attribute)
      target_variables = df['status'].unique() 
      status_check = {}
      for i in target_variables:
          status_check[i] = single_probability(df,'age',age,i) * single_probability(df,'salary',salary,i) * single_probability(df,'department',department_attribute,i)
      print(status_check)
def_status(df,'systems',28,50)

{'senior': 0.0, 'junior': 0.01796455325640015}
